<a href="https://colab.research.google.com/github/Kaiser2110/MateriaIA/blob/main/tarea4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [25]:

data = pd.read_csv("AAPL.csv")
data['Date'] = pd.to_datetime(data['Date'])
data['mt'] = data['Date'].apply(lambda x: str(x.month))
# add time index
data["time_idx"] = data["Date"].dt.year * 12 + data["Date"].dt.month
data["time_idx"] -= data["time_idx"].min()


data.sample(10, random_state=521)

,Date,Open,High,Low,Close,Adj Close,Volume,mt,time_idx
8649,2015-03-31,31.522499,31.622499,31.090000,31.107500,28.030130,168362400,3,411
2607,1991-04-08,0.618304,0.625000,0.613839,0.625000,0.503549,72475200,4,124
2981,1992-09-28,0.401786,0.401786,0.390625,0.399554,0.326377,149520000,9,141
2188,1989-08-09,0.392857,0.408482,0.391741,0.392857,0.310474,195160000,8,104
4403,1998-05-14,0.271205,0.271763,0.265625,0.268415,0.228137,162680000,5,209
1786,1988-01-07,0.388393,0.399554,0.379464,0.397321,0.309820,212800000,1,85
3623,1995-04-12,0.341518,0.353795,0.333705,0.348214,0.293426,474712000,4,172
4488,1998-09-15,0.328125,0.344308,0.325893,0.340960,0.289797,433652800,9,213
10456,2022-06-02,147.830002,151.270004,146.860001,151.210007,150.312454,72348100,6,498
9362,2018-01-29,42.540001,42.540001,41.767502,41.990002,39.812344,202561600,1,445


In [26]:
max_prediction_length = 6
max_encoder_length = 24
training_cutoff = data["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Close",
    group_ids=["mt"],
    min_encoder_length=max_encoder_length // 2,  # mantenga la longitud del codificador larga (como está en el conjunto de validación)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["Open", "High", "Low", "Adj Close", "Volume"],
    time_varying_unknown_categoricals=[],
    target_normalizer=GroupNormalizer(
        groups=["mt"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

# crear un conjunto de validación (predict=True) que significa predecir los últimos puntos max_prediction_length en el tiempo
# para cada serie
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# crea dataloaders para el modelo
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=4)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [28]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

trainer = pl.Trainer(
    max_epochs=100, 
    gradient_clip_val=0.05,
    limit_train_batches=30,
    callbacks=[lr_logger, early_stop_callback],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


Number of parameters in network: 83.8k


In [47]:
compiled_model = torch.compile(tft)
trainer.fit(
    compiled_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,
)

INFO: 
   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 288   
4  | static_variable_selection          | VariableSelectionNetwork        | 5.7 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 11.9 K
6  | decoder_variable_selection         | VariableSelectionNetwork        | 11.9 K
7  | static_context_variable_selection  | GatedResidualNetwork            | 4.3 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 4.3 K 
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Process ForkProcess-25:
Process ForkProcess-26:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/pyt